In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Load the persisted ChromaDB vector store
embedding = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)
persist_directory = "./db"

vector_database = Chroma(
    collection_name="local-rag",
    persist_directory=persist_directory,
    embedding_function=embedding
)
print('database loaded')

database loaded


In [3]:
# Initialize the Ollama LLaMA 3.1 model
llm = Ollama(model="llama3.1")

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model specialized in physics. Your task is to reformulate the following question into five different versions to retrieve the most relevant physics documents.
    Original question: {question}""",
)

retriever = MultiQueryRetriever.from_llm(
    vector_database.as_retriever(search_kwargs={"k": 3}),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """You are a helpful assistant trained to answer physics questions based on the provided context.
Use only the context below to answer the following question as clearly as possible:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [4]:
def post_process_answer(answer):
    # Ensure the answer is focused on the physics topic
    # Optionally trim any irrelevant parts or hallucinated information
    processed_ans =  answer.split("Answer:")[-1].strip()
    return processed_ans if processed_ans else "No relevant info found in the context..."


In [5]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
def getAns(query):
    answer = chain.invoke(query)
    final_ans = post_process_answer(answer)
    return query+'\n'+final_ans

In [7]:
print(getAns(input()))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


Rotational motion refers to the movement of an object around a fixed axis, resulting in a change in its orientation or position without necessarily changing its location. This type of motion involves rotation about a pivot point or fulcrum and can be described using parameters such as angular displacement, velocity, acceleration, and torque.

Key characteristics of rotational motion include:

1. **Rotation**: The object moves around an axis, which is perpendicular to the plane of motion.
2. **Angular Displacement (θ)**: Measures how much the object has turned from its initial position.
3. **Angular Velocity (ω)**: Describes the rate at which the object rotates; it's often measured in radians per second.
4. **Rotational Speed**: Refers to the speed of rotation, usually expressed as revolutions per minute (RPM) or revolutions per second (RPS).
5. **Torque**: Measures the twisting force that causes the rotational motion.

Examples of rotational motion include:

- A child spinning around o